In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
## local 환경에서 실행
articles = pd.read_csv("../data/articles.csv")
orders_csv = pd.read_csv("../data/orders.csv")
products = pd.read_csv("../data/products.csv")

C:\Users\y0010\AppData\Local\Temp\ipykernel_23148\394980597.py:2: DtypeWarning: Columns (14,15,23,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  articles = pd.read_csv("../data/articles.csv")
C:\Users\y0010\AppData\Local\Temp\ipykernel_23148\394980597.py:3: DtypeWarning: Columns (5,7,26) have mixed types. Specify dtype option on import or set low_memory=False.
  orders_csv = pd.read_csv("../data/orders.csv")


In [4]:
articles_df= pd.DataFrame(articles).iloc[:,1:]
orders_df = pd.DataFrame(orders_csv).iloc[:,1:]
products_df = pd.DataFrame(products).iloc[:,1:]

# orders table

In [5]:
# product와 연결되는 item product_id를 찾는 함수
def find_product(index):
    '''
    Find out the product_id in orders table with index number
    
    Args:
    index(int): input the index
    
    Returns:
    product_id
    '''
    try:
        product_id = str(eval(orders_csv["items"][index])[0]['product_id'])
        return product_id
    except:
        print("no item")

In [7]:
# column에 product_id 포함하는지 검색, customer_id 특정 가능
def find_product_id(df,column,product_id,customer_id=None):
    '''
    Find if the dataframe contains the product_id and returns the dataframe which contains input product_id and customer_id
    
    Args:
    df(Dataframe): orders table
    column(column): 'item' column
    product_id(string): product_id which you want figure out
    customer_id(string): customer_id which you want figure out, defalt=None
    
    Returns:
    df(Dataframe): table
    '''
    if customer_id == None:
        df_temp = df[df[column].str.contains(product_id ,na=False)]
    else:
        df_temp = df[( df[column].str.contains(product_id ,na=False) ) & ( df['customer_id'].str.contains(customer_id ,na=False) )]
    return df_temp
# find_product_id(orders_df,'items','5f51fecca0b91001bf48f211','60c6f4b80ccc56001bc8075f')

In [8]:
def strtodict(string):
    return eval(string)

In [9]:
# 전처리 할때 딕셔너리 형태의 str null 값 찾기
def findnull(df,column):
    df[column] = df[column].apply(strtodict)
    return df[df[column].str.len() == 0]

# log table

In [10]:
path = '../data/1663027200000/'
import os
def logconcat(path, startswith):
    file_list = os.listdir(path)
    file_list_py = [file for file in file_list if file.startswith(startswith)]
    file_list_py
    df = pd.DataFrame()
    for i in file_list_py:
        data = pd.read_json(path+i,lines=True)
        df = pd.concat([df,data])
    return df.reset_index(drop = True)

In [12]:
# logconcat(path,'1')

In [13]:
def findcontext(df):
    '''
    Figure out the context column table's dictionary which is 'page' value
    
    Args:
    df(Dataframe): log dataset
    
    Returns:
    li(list): values of 'page'
    '''
    li = []
    for i, row in df.iterrows():
        for j in row['context'].keys():
            if j == 'page':
                li.append(row['context'][j])
    return li

In [14]:
def contexttocolumn(df):
    '''
    Add columns with context column's 'page' key-value
    
    Args:
    df(Dataframe): log table
    
    Returns:
    df(Dataframe): key-value added log table
    
    '''
    li = []
    for i, row in df.iterrows():
        for j in row['context'].keys():
            if j == 'page':
                li.append(row['context'][j])    
    df_dic = df.copy()
    df_dic['page'] = li
    path, referrer, search, title, url = [], [],[],[],[]
    for i, row in df_dic.iterrows():
        for j in row['page'].keys():
            if j == 'path':
                path.append(row['page'][j])
            if j == 'referrer':
                referrer.append(row['page'][j])
            if j == 'search':
                search.append(row['page'][j])
            if j == 'title':
                title.append(row['page'][j])
            if j == 'url':
                url.append(row['page'][j])
    df_dic['path'] = path
    df_dic['referrer'] = referrer
    df_dic['search'] = search
    df_dic['title'] = title
    df_dic['url'] = url  
    return df_dic

In [61]:
len(df_dic[df_dic['traits'].isna() == True]) / len(df_dic)

0.5500329404875192

In [62]:
len(df_dic[df_dic['userId'].isna() == True]) / len(df_dic)

0.004355464460874021

In [16]:
# traits에 customerId인 데이터 찾기
def find_data_customerId(df,column,customerId):
    '''
    Make the dataframe which contains input customerId
    
    Args:
    df(Dataframe): log dataframe 
    column(String): 'trait' column
    customerId(String): customerId you want to find
    
    Returns:
    df(Dataframe): Dataframe which contains input customerId
    '''
    li=[]
    for i, row in df.iterrows():
        try:
            if row[column]['customerId'] == customerId:
                li.append(i)
        except:
            continue
    return df.iloc[li,:]
# find_data_customerId(df_dic,'traits','5d705e9419efa30eb29141e8')